In [6]:
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import theano
from PIL import Image
from numpy import *



# input image dimensions
img_rows, img_cols = 32,32

# number of channels
img_channels =3


#  data

path1 = 'C:/Users/dundeva/Documents/CroppedImages'    #path of folder of images    
path2 = 'C:/Users/dundeva/Documents/AugmentedImages/data/train'  #path of folder of saved images    

listing = os.listdir(path1) 
num_samples=size(listing)
print num_samples


for file in listing:
    im = Image.open(path1 + '//' + file)   
    img = im.resize((img_rows,img_cols))
    gray = img.convert('L')
                           
    gray.save(path2 +'//' +  file, "JPEG")

imlist = os.listdir(path1)

im1 = array(Image.open('C:/Users/dundeva/Documents/AugmentedImages/data/train' + '//'+ imlist[0])) # open one image to get size
m,n = im1.shape[0:2] # get the size of the images
imnbr = len(imlist) # get the number of images

# create matrix to store all flattened images
immatrix = array([array(Image.open('C:/Users/dundeva/Documents/AugmentedImages/data/train'+ '//' + im2)).flatten()
              for im2 in imlist],'f')
                
label=np.ones((num_samples,),dtype = int)
label[0:129]=0#Anger
label[129:136]=1#Contempt
label[136:168]=2#Disgust
label[168:211]=3#Fear
label[211:2812]=4#Happiness
label[2812:6393]=5#Nuetral
label[6393:6505]=6#Sadness
label[6505:6704]=7#Surprise


data,Label = shuffle(immatrix,label, random_state=2)
train_data = [data,Label]
'''
img=immatrix[167].reshape(img_rows,img_cols)
plt.imshow(img)
plt.imshow(img,cmap='gray')
print (train_data[0].shape)
print (train_data[1].shape)
'''


#batch_size to train
batch_size = 128
# number of output classes
nb_classes =10
# number of epochs to train
nb_epoch =10
#data_augmentation = True

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 3
# convolution kernel size
nb_conv = 3

(X, y) = (train_data[0],train_data[1])


# split X and y into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)


X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same',input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
"""
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

"""
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))


#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='RMSprop',
              metrics=["accuracy"])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])



6711
('X_train shape:', (5368L, 1L, 32L, 32L))
(5368L, 'train samples')
(1343L, 'test samples')
Train on 5368 samples, validate on 1343 samples
Epoch 1/10
5368/5368 [==============================] - 67s - loss: 1.1639 - acc: 0.4950 - val_loss: 0.9584 - val_acc: 0.5369
Epoch 2/10
5368/5368 [==============================] - 65s - loss: 0.8963 - acc: 0.6375 - val_loss: 0.9600 - val_acc: 0.5756
Epoch 3/10
5368/5368 [==============================] - 63s - loss: 0.7677 - acc: 0.7139 - val_loss: 0.7114 - val_acc: 0.7461
Epoch 4/10
5368/5368 [==============================] - 63s - loss: 0.6489 - acc: 0.7686 - val_loss: 0.7281 - val_acc: 0.7193
Epoch 5/10
5368/5368 [==============================] - 66s - loss: 0.6078 - acc: 0.7876 - val_loss: 0.7088 - val_acc: 0.7461
Epoch 6/10
5368/5368 [==============================] - 66s - loss: 0.5336 - acc: 0.8130 - val_loss: 0.6410 - val_acc: 0.7796
Epoch 7/10
5368/5368 [==============================] - 67s - loss: 0.5036 - acc: 0.8212 - val_loss: